In [2]:
import mph

In [3]:
client = mph.start()
client.load("data/bend/beam.mphbeam.mph")

Could not locate any Comsol installation.


RuntimeError: Could not locate any Comsol installation.

In [7]:
mph.discovery.search_disk()

[]

In [6]:
import platform                        # regular expressions
from pathlib import Path  
import re
platform.system()
locations = [
            Path('/Applications'),
            Path.home() / 'Applications',
        ]

In [7]:
folders = [item for location in locations
                    if location.is_dir()
                    for item in location.iterdir()
                    if item.is_dir() and re.match('(?i)comsol', item.name)]

In [8]:
folders

[PosixPath('/Applications/COMSOL Multiphysics.app'),
 PosixPath('/Applications/COMSOL61')]

In [ ]:
folder